# Deteksi URL Phishing dengan menggunakan metode Machine Learning: Random Forest

In [13]:
import pandas as pd
from urllib.parse import urlparse,urlencode
import re

## **1. Data yang Digunakan**
Data yang digunakan pada project ini adalah url yang sah (0) dan yang phishing (1).
Untuk data url yang dikategorikan sebagai phishing dapat diambil dari data opensource [PhishTank](https://www.phishtank.com/developer_info.php). Sedangkan untuk data url yang sah diambil dari [University of New Brunswick](https://www.unb.ca/cic/datasets/url-2016.html).

In [14]:
#Membaca data url legitimate
data0 = pd.read_csv("dataset/Benign_list_big_final.csv")
data0.columns = ['URLs']
# legiturl = data0.sample(n = 10000, random_state = 12).copy()
legiturl = data0.copy()
legiturl = legiturl.reset_index(drop=True)

In [15]:
#Membaca data url phishing
data1 = pd.read_csv("dataset/online-valid.csv")
data1 = data1[['url']]
phishurl = data1.copy()
phishurl = phishurl.reset_index(drop=True)

In [16]:
phishurl

,url
0,http://u1047531.cp.regruhosting.ru/acces-inges...
1,http://hoysalacreations.com/wp-content/plugins...
2,http://www.accsystemprblemhelp.site/checkpoint...
3,http://www.accsystemprblemhelp.site/login_atte...
4,https://firebasestorage.googleapis.com/v0/b/so...
...,...
14853,http://bancoestado700.blogspot.com/
14854,http://www.habbocreditosparati.blogspot.com/
14855,http://creditiperhabbogratissicuro100.blogspot...
14856,http://mundovirtualhabbo.blogspot.com/2009_01_...


## **2. Ekstrasi Fitur**

Pada tahap ini akan dilakukan ektrasi fitur yang berasal dari dataset URL.
Adapun fitur yang dapat diekstraksi pada kategori ini adalah sebagai berikut:

*   URL Domain
*   URL IP Address
*   Simbol "@" pada URL
*   Panjang URL
*   Kedalaman URL
*   Tanda "//" pada URL
*   "http/https" pada nama domain
*   Layanan penyingkat URL seperti “TinyURL”
*   Prefix atau Suffix "-" pada domain

In [17]:
#list penyingkat domain
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [18]:
# 1.URL Domain
def getDomain(url):
    '''
    Fungsi untuk mendapatkan nama domain
    '''
    domain = urlparse(url).netloc
    if re.match(r"^www.",domain):
        domain = domain.replace("www.","")
    return domain

# 2.Cek apakah ada IP address
def havingIP(url):
    '''
    Fungsi untuk mengetahui apakah URL menggunakan IP address atau tidak
    Website yang terindikasi phishing memakai URL berupa IP address 
    '''
    try:
        ipaddress.ip_address(url)
        ip = 1
    except:
        ip = 0
    return ip

# 3.Cek apakah ada simbol @ pada URL
def haveAtSign(url):
    '''
    Fungsi untuk mengetahui apakah terdapat simbol @ pada URL
    Jika terdapat simbol @ maka URL terindikasi phishing
    '''
    if "@" in url:
        at = 1    
    else:
        at = 0    
    return at

# 4.Mengetahui panjang URL
def getLength(url):
    '''
    Fungsi untuk mengetahui panjang URL
    URL yang legit memiliki panjang kurang dari 54 karakter'''
    if len(url) < 54:
        length = 0            
    else:
        length = 1            
    return length

# 5.Mengetahui kedalaman URL
def getDepth(url):
    '''
    Fungsi untuk mengetahui kedalaman suatu URL
    Kedalaman URL ditandai dengan berapa banyak simbol /'''
    s = urlparse(url).path.split('/')
    depth = 0
    for j in range(len(s)):
        if len(s[j]) != 0:
            depth = depth+1
    return depth

# 6.Cek '//' pada URL
def redirection(url):
    '''
    Fungsi untuk mengetahui keberadaan simbol // pada URL
    Simbol // digunakan untuk menuju website lain, pada kasus URL pishing simbol // terdapat pada posisi 6 atau 7 pada URL
    '''
    pos = url.rfind('//')
    if pos > 6:
        if pos > 7:
            return 1
        else:
            return 0
    else:
        return 0
    
# 7.Apakah URL menggunakan HTTPS atau HTTP
def httpDomain(url):
    '''
    Fungsi untuk mengetahui keberadaan kata https pada domain URL
    Biasanya URL pishing menambahkan https pada domain untuk menipu user
    '''
    domain = urlparse(url).netloc
    if 'https' in domain:
        return 1
    else:
        return 0
    
# 8. Cek apakah menggunakan penyingkat domain
def tinyURL(url):
    '''
    Fungsi untuk mengetahui apakah URL menggunakan penyingkat domain
    Daftar penyedia singkatan tersedia pada list shortening_url, biasanya URL terindikasi phishing menggunakan penyingkat
    '''
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

# 9.Cek apakah ada tanda '-' pada domain
def prefixSuffix(url):
    '''
    Fungsi untuk mengetahui keberadaan simbol - pada domain URL
    URL yang memiliki simbol - pada domain terindikasi sebagain phising'''
    if '-' in urlparse(url).netloc:
        return 1            # phishing
    else:
        return 0            # legitimate

## 3. Menggabungkan Ekstraksi Fitur

In [19]:
def featureExtraction(url,label):

    features = []
    
    features.append(getDomain(url))
    features.append(havingIP(url))
    features.append(haveAtSign(url))
    features.append(getLength(url))
    features.append(getDepth(url))
    features.append(redirection(url))
    features.append(httpDomain(url))
    features.append(tinyURL(url))
    features.append(prefixSuffix(url))
    features.append(label)

    return features

## 4. Ekstraksi Fitur pada Data Legitimate dan Phishing URL 

In [20]:
legit_data = []
legit_label = 0
for i in range(legiturl.shape[0]):
        url = legiturl.iloc[i, 0]
        legit_data.append(featureExtraction(url, legit_label))

In [21]:
phish_data = []
phish_label = 1
for i in range(phishurl.shape[0]):
        url = phishurl.iloc[i, 0]
        phish_data.append(featureExtraction(url, phish_label))

In [22]:
#converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection', 
                 'https_Domain', 'TinyURL', 'Prefix/Suffix', 'Label']

legitimate = pd.DataFrame(legit_data, columns= feature_names)
phishing = pd.DataFrame(phish_data, columns= feature_names)

In [23]:
legitimate

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,Label
0,1337x.to,0,0,1,3,0,0,0,0,0
1,1337x.to,0,0,1,3,0,0,0,0,0
2,1337x.to,0,0,1,3,0,0,0,0,0
3,1337x.to,0,0,1,3,0,0,0,0,0
4,1337x.to,0,0,1,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
35372,lastpass.com,0,0,1,1,0,0,0,0,0
35373,lastpass.com,0,0,1,1,0,0,0,0,0
35374,lastpass.com,0,0,1,1,0,0,0,0,0
35375,lastpass.com,0,0,1,1,0,0,0,0,0


## 5. Menyatukan 2 Dataset Legitimate and Phishing

In [24]:
urldata = pd.concat([legitimate, phishing]).reset_index(drop=True)
urldata

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,Label
0,1337x.to,0,0,1,3,0,0,0,0,0
1,1337x.to,0,0,1,3,0,0,0,0,0
2,1337x.to,0,0,1,3,0,0,0,0,0
3,1337x.to,0,0,1,3,0,0,0,0,0
4,1337x.to,0,0,1,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
50230,bancoestado700.blogspot.com,0,0,0,0,0,0,1,0,1
50231,habbocreditosparati.blogspot.com,0,0,0,0,0,0,1,0,1
50232,creditiperhabbogratissicuro100.blogspot.com,0,0,1,3,0,0,1,0,1
50233,mundovirtualhabbo.blogspot.com,0,0,1,1,0,0,1,0,1


In [25]:
# Menyimpan data url yang sudah di ekstrak dalam format CSV
urldata.to_csv('dataset/alldata.csv', index=False)